### 文本增强

#### Synthetic comments（基于马尔科夫链随机生成）

In [ ]:
train_data=pd.read_csv('C:/Users/25529/Downloads/jigsaw/train_data.csv', encoding='ANSI')
toxic_data=train_data[train_data.toxic==1]

In [ ]:
import markovify as mk
doc = train_data.loc[train_data.toxic==1, 'comment_text'].tolist()
text_model = mk.Text(doc)
for i in range(10):
    print(text_model.make_sentence())

Hello, Nawlins Why do you keep fucking with people on Wikipedia by itself is BIASED!
I've got a pussy ass pansy.
Drive them into the fucking talk to prejudice people I know.
I'm trying to tell you that have gotten raped by filthy pigs? which one is a moron in an Oxymoron Oxymoron is a greedy bastard who wants to talk to you or your allies were interested in your mother ass and pussy i want to know a father talks to a party, you always go around after other editors suck enormous amounts of vandalism attacks happening, and you know anything about a blog Why is it knowing you will not be given the fact they exist violate any copyright?
I even suggested re-editing in the sinking ship USA...and it will be blocked and from Heron to Brunel, if thats racist, then sue me.
Leave useful edit summaries are encouraged.
== f.uck you == Brothers and sisters, In this article, but will he get so angry when I am a cunt.
In case you haven't got any shame what is Modi doing about it?
All good things, of c

#### Synonym Replacement（同义词替换）

In [ ]:
from nltk.corpus import stopwords
import random
from nltk.corpus import wordnet
def get_synonyms(word):
    
    synonyms = set()
    
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            synonym = l.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym) 
    if word in synonyms:
        synonyms.remove(word)
    
    return list(synonyms)

In [ ]:
stop_words = []
for w in stopwords.words('english'):
    stop_words.append(w)

In [ ]:
def synonym_replacement(words, n):
    
    words = words.split()
    
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word not in stop_words]))
    random.shuffle(random_word_list)
    num_replaced = 0
    
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        
        # n为随机替换单词数
        if num_replaced >= n: 
            break

    sentence = ' '.join(new_words)

    return sentence

In [ ]:
trial_sent =np.array(toxic_data.comment_text)[10]
print(trial_sent)

for n in range(3):
    print(f" Example of Synonym Replacement: {synonym_replacement(trial_sent,n)}")

Hell, no. I ain't serving the war, I was helping people, This is fucked up.
 Example of Synonym Replacement: Hell, no. I ain't serving the war, I was helping people, This is fucked up.
 Example of Synonym Replacement: Hell, ordinal number I ain't serving the war, I was helping people, This is fucked up.
 Example of Synonym Replacement: Hell, no. i ain't serving the war, i was helping people, This is know up.


In [ ]:
toxic_data.insert(toxic_data.shape[1], 'SR', 0)
for i in range(len(np.array(toxic_data.comment_text))):
    toxic_data['SR'][i:i+1]=synonym_replacement(np.array(toxic_data.comment_text)[i],3)
toxic_data.head()

C:\Users\25529\AppData\Local\Temp\ipykernel_21792\2831844676.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_data['SR'][i:i+1]=synonym_replacement(np.array(toxic_data.comment_text)[i],3)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._set_values(indexer, value)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,SR
15802,44cc46e639717428,LOL - you thought your sources - including the...,1,0,1,0,0,0,LOL - you thought your germ - including the TB...
15803,44cc5d68e400d503,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! I CAN DO ...,1,0,1,0,0,0,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! ace CAN b...
15826,44d8a1aad4973721,"Hot4\nMy Ryan, do come back to me you gorgeous...",1,0,0,0,0,0,"Hot4 My Ryan, do seed back to me you gorgeous ..."
15833,44ddd41a39cf01c5,How about you fuck off and don't stalk my edit...,1,0,1,0,0,0,How about you shtup off and don't stem my edit...
15839,44e225b1139aa592,"THERE HASNT BEEN ANY EPISODES SINCE MARCH 7, D...",1,0,1,0,0,0,THERE HASNT exist whatever episode SINCE MARCH...


#### Random Deletion（随机删除）

In [ ]:
import random
def random_deletion(words, p):

    words = words.split()
    if len(words) == 1:
        return words

    #以概率p随机删除
    new_words = []
    for word in words:
        r = random.uniform(0, 1)
        if r > p:
            new_words.append(word)
            
    #删除所有词语
    if len(new_words) == 0:
        rand_int = random.randint(0, len(words)-1)
        return [words[rand_int]]

    sentence = ' '.join(new_words)
    
    return sentence

In [ ]:
print(random_deletion(trial_sent,0.2))
print(random_deletion(trial_sent,0.3))
print(random_deletion(trial_sent,0.4))

Hell, no. ain't serving war, I This fucked
no. war, I people, This is fucked
ain't serving the was people, This is fucked


In [ ]:
toxic_data.insert(toxic_data.shape[1], 'RD', 0)
for i in range(len(np.array(toxic_data.comment_text))):
    toxic_data['RD'][i:i+1]=random_deletion(np.array(toxic_data.comment_text)[i],0.3)
toxic_data.head()

C:\Users\25529\AppData\Local\Temp\ipykernel_21792\4266470768.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_data['RD'][i:i+1]=random_deletion(np.array(toxic_data.comment_text)[i],0.3)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._set_values(indexer, value)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ret

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,SR,RD
15802,44cc46e639717428,LOL - you thought your sources - including the...,1,0,1,0,0,0,LOL - you thought your germ - including the TB...,LOL your sources - the TBR more reliable News....
15803,44cc5d68e400d503,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! I CAN DO ...,1,0,1,0,0,0,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! ace CAN b...,HAHA THIS IS FUCKIN DO WHATEVER I WANT!!!!!!
15826,44d8a1aad4973721,"Hot4\nMy Ryan, do come back to me you gorgeous...",1,0,0,0,0,0,"Hot4 My Ryan, do seed back to me you gorgeous ...",Hot4 come back me you gorgeous piece sex
15833,44ddd41a39cf01c5,How about you fuck off and don't stalk my edit...,1,0,1,0,0,0,How about you shtup off and don't stem my edit...,about you and don't stalk my 201.215.187.159
15839,44e225b1139aa592,"THERE HASNT BEEN ANY EPISODES SINCE MARCH 7, D...",1,0,1,0,0,0,THERE HASNT exist whatever episode SINCE MARCH...,"THERE HASNT BEEN SINCE MARCH 7, 50.180.208.181"


#### Random Swap（随机交换）

In [ ]:
def swap_word(new_words):
    
    random_idx_1 = random.randint(0, len(new_words)-1)
    random_idx_2 = random_idx_1
    counter = 0
    
    while random_idx_2 == random_idx_1:
        random_idx_2 = random.randint(0, len(new_words)-1)
        counter += 1
        
        if counter > 3:
            return new_words
    
    new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1] 
    return new_words

def random_swap(words, n):
    
    words = words.split()
    new_words = words.copy()
    # n为随机交换单词数
    for _ in range(n):
        new_words = swap_word(new_words)
        
    sentence = ' '.join(new_words)
    
    return sentence

In [ ]:
print(random_swap(trial_sent,1))
print(random_swap(trial_sent,2))
print(random_swap(trial_sent,3))

Hell, no. is ain't serving the war, I was helping people, This I fucked up.
Hell, no. I is serving the war, I was people, helping This ain't fucked up.
I no. people, ain't serving the war, I was helping is This Hell, fucked up.


In [ ]:
toxic_data.insert(toxic_data.shape[1], 'RS', 0)
for i in range(len(np.array(toxic_data.comment_text))):
    toxic_data['RS'][i:i+1]=random_swap(np.array(toxic_data.comment_text)[i],3)
toxic_data.head()

C:\Users\25529\AppData\Local\Temp\ipykernel_21792\1945283172.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_data['RS'][i:i+1]=random_swap(np.array(toxic_data.comment_text)[i],3)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._set_values(indexer, value)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,SR,RD,RS
15802,44cc46e639717428,LOL - you thought your sources - including the...,1,0,1,0,0,0,LOL - you thought your germ - including the TB...,LOL your sources - the TBR more reliable News....,LOL - News. thought your was - including the T...
15803,44cc5d68e400d503,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! I CAN DO ...,1,0,1,0,0,0,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! ace CAN b...,HAHA THIS IS FUCKIN DO WHATEVER I WANT!!!!!!,HAHA THIS WHATEVER WANT!!!!!! FUCKIN PAGE!!!!!...
15826,44d8a1aad4973721,"Hot4\nMy Ryan, do come back to me you gorgeous...",1,0,0,0,0,0,"Hot4 My Ryan, do seed back to me you gorgeous ...",Hot4 come back me you gorgeous piece sex,"Hot4 gorgeous Ryan, do back come to me you of ..."
15833,44ddd41a39cf01c5,How about you fuck off and don't stalk my edit...,1,0,1,0,0,0,How about you shtup off and don't stem my edit...,about you and don't stalk my 201.215.187.159,off How you fuck about and don't edits? my sta...
15839,44e225b1139aa592,"THERE HASNT BEEN ANY EPISODES SINCE MARCH 7, D...",1,0,1,0,0,0,THERE HASNT exist whatever episode SINCE MARCH...,"THERE HASNT BEEN SINCE MARCH 7, 50.180.208.181",THERE BEEN SINCE ANY EPISODES HASNT MARCH DUMB...


#### Random Insertion（随机插入）

In [ ]:
def random_insertion(words, n):
    
    words = words.split()
    new_words = words.copy()
    
    for _ in range(n):
        add_word(new_words)
        
    sentence = ' '.join(new_words)
    return sentence

def add_word(new_words):
    
    synonyms = []
    counter = 0
    
    while len(synonyms) < 1:
        random_word = new_words[random.randint(0, len(new_words)-1)]
        synonyms = get_synonyms(random_word)
        counter += 1
        if counter >= 10:
            return
        
    random_synonym = synonyms[0]
    random_idx = random.randint(0, len(new_words)-1)
    new_words.insert(random_idx, random_synonym)

In [ ]:
print(random_insertion(trial_sent,1))
print(random_insertion(trial_sent,2))
print(random_insertion(trial_sent,3))

Hell, cost no. I ain't serving the war, I was helping people, This is fucked up.
 Hell, no. I cost ain't serving the war, I was helping people, This is fucked up.
Hell, cost no. I  ain't serving the war, I was helping people, This  is fucked up.


In [ ]:
toxic_data.insert(toxic_data.shape[1], 'RI', 0)
for i in range(len(np.array(toxic_data.comment_text))):
    toxic_data['RI'][i:i+1]=random_insertion(np.array(toxic_data.comment_text)[i],3)
toxic_data.head()

C:\Users\25529\AppData\Local\Temp\ipykernel_21792\3123746593.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  toxic_data['RI'][i:i+1]=random_insertion(np.array(toxic_data.comment_text)[i],3)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1105: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._set_values(indexer, value)
c:\Users\25529\anaconda3\lib\site-packages\pandas\core\series.py:1056: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,SR,RD,RS,RI
15802,44cc46e639717428,LOL - you thought your sources - including the...,1,0,1,0,0,0,LOL - you thought your germ - including the TB...,LOL your sources - the TBR more reliable News....,LOL - News. thought your was - including the T...,LOL - you thought your sources - including the...
15803,44cc5d68e400d503,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! I CAN DO ...,1,0,1,0,0,0,HAHA THIS IS MY FUCKIN PAGE!!!!!!!!! ace CAN b...,HAHA THIS IS FUCKIN DO WHATEVER I WANT!!!!!!,HAHA THIS WHATEVER WANT!!!!!! FUCKIN PAGE!!!!!...,HAHA THIS cost IS MY FUCKIN PAGE!!!!!!!!! I C...
15826,44d8a1aad4973721,"Hot4\nMy Ryan, do come back to me you gorgeous...",1,0,0,0,0,0,"Hot4 My Ryan, do seed back to me you gorgeous ...",Hot4 come back me you gorgeous piece sex,"Hot4 gorgeous Ryan, do back come to me you of ...","Hot4 make My Ryan, do come back to make out me..."
15833,44ddd41a39cf01c5,How about you fuck off and don't stalk my edit...,1,0,1,0,0,0,How about you shtup off and don't stem my edit...,about you and don't stalk my 201.215.187.159,off How you fuck about and don't edits? my sta...,dispatch How about you fuck off still hunt and...
15839,44e225b1139aa592,"THERE HASNT BEEN ANY EPISODES SINCE MARCH 7, D...",1,0,1,0,0,0,THERE HASNT exist whatever episode SINCE MARCH...,"THERE HASNT BEEN SINCE MARCH 7, 50.180.208.181",THERE BEEN SINCE ANY EPISODES HASNT MARCH DUMB...,any THERE HASNT BEEN ANY any EPISODES SINCE MA...


#### 扩充样本量

In [ ]:
train_data = train_data[['comment_text', 'toxic']]
train_data

,comment_text,toxic
0,if there is a chromosone then e=what is it?Sma...,0
1,Hollywood Undead \n\nI have collected articles...,0
2,"""\n\n Rollback \n\nI've enabled rollback on yo...",0
3,Another backlog. Thanks. (Trouble?/My Work),0
4,""" - unsigned\n\nWe do include it. This article...",0
...,...,...
213371,":Jerome, I see you never got around to this…! ...",0
213372,==Lucky bastard== \n http://wikimediafoundatio...,0
213373,==shame on you all!!!== \n\n You want to speak...,0
213374,MEL GIBSON IS A NAZI BITCH WHO MAKES SHITTY MO...,1


In [ ]:
train_data = train_data.append(toxic_data[['SR', 'toxic']].rename(columns={'SR': 'comment_text'}))
train_data = train_data.append(toxic_data[['RD', 'toxic']].rename(columns={'RD': 'comment_text'}))
train_data = train_data.append(toxic_data[['RS', 'toxic']].rename(columns={'RS': 'comment_text'}))
train_data = train_data.append(toxic_data[['RI', 'toxic']].rename(columns={'RI': 'comment_text'}))
train_data

,comment_text,toxic
0,if there is a chromosone then e=what is it?Sma...,0
1,Hollywood Undead \n\nI have collected articles...,0
2,"""\n\n Rollback \n\nI've enabled rollback on yo...",0
3,Another backlog. Thanks. (Trouble?/My Work),0
4,""" - unsigned\n\nWe do include it. This article...",0
...,...,...
213263,am sorry for being a dickhead! I cannae help i...,1
213275,NIGEL cost adenine IS A CRAZY cost IDIOT!!!,1
213343,==Fourth Baldrick possibly being cleverer than...,1
213360,""" == IRAN == That’s right, Iran. It was our dr...",1


#### BERT评测

In [ ]:
data = train_data
#提取语句并处理
sentencses=['[CLS] ' + sent + ' [SEP]' for sent in data.comment_text.values]
labels=data.toxic.values
print("第一句话:",sentencses[0])
tokenizer=BertTokenizer.from_pretrained(bert_pre_tokenizer,do_lower_case=True)
tokenized_sents=[tokenizer.tokenize(sent) for sent in sentencses]
print("tokenized的第一句话:",tokenized_sents[0])

第一句话: [CLS] if there is a chromosone then e=what is it?Small Text [SEP]
tokenized的第一句话: ['[CLS]', 'if', 'there', 'is', 'a', 'ch', '##rom', '##oso', '##ne', 'then', 'e', '=', 'what', 'is', 'it', '?', 'small', 'text', '[SEP]']


In [ ]:
#定义句子最大长度
MAX_LEN=96
#将分割后的句子转化成数字  word-->idx
input_ids=[tokenizer.convert_tokens_to_ids(sent) for sent in tokenized_sents]
print("转化后的第一个句子:",input_ids[0])

转化后的第一个句子: [101, 2065, 2045, 2003, 1037, 10381, 21716, 19137, 2638, 2059, 1041, 1027, 2054, 2003, 2009, 1029, 2235, 3793, 102]


In [ ]:
#PADDING
import keras
input_ids=keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print("Padding 第一个句子:",input_ids[0])

Padding 第一个句子: [  101  2065  2045  2003  1037 10381 21716 19137  2638  2059  1041  1027
  2054  2003  2009  1029  2235  3793   102     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]


In [ ]:
#mask
attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
print("第一个attention mask:",attention_masks[0])

第一个attention mask: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [ ]:
#划分训练集、验证集
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels,
                                                            random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids,
                                             random_state=2018, test_size=0.1)
print("训练集的一个inputs",train_inputs[0])
print("训练集的一个mask",train_masks[0])

训练集的一个inputs [  101  1045  4384  3158  9305 22556  2006 13292  1012  2322  2008  2081
  2033  2514  2009  2052  5258  2153  1012   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
训练集的一个mask [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [ ]:
#将训练集、验证集转化成tensor
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

#生成dataloader
batch_size = 16
train_data = Data.TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = Data.RandomSampler(train_data)
train_dataloader = Data.DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
validation_data = Data.TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = Data.SequentialSampler(validation_data)
validation_dataloader = Data.DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [ ]:
modelConfig = BertConfig.from_pretrained(bert_config)
model = BertForSequenceClassification.from_pretrained(bert_pre_model, config=modelConfig)
print(model.cuda())

Some weights of the model checkpoint at C:/Users/25529/Downloads/bert-base-uncased/pytorch_model.bin were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were n

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]

optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [ ]:
#定义一个计算准确率的函数
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
#训练开始
train_loss_set = []#可以将loss加入到列表中，后期画图使用
epochs = 3
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")#判断CUDA是否能使用，不可以就使用CPU
for _ in range(epochs):
    #训练开始
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        optimizer.zero_grad()
        #取第一个位置，BertForSequenceClassification第一个位置是Loss，第二个位置是[CLS]的logits
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)[0]
        train_loss_set.append(loss.item())
        loss.backward()
        optimizer.step()
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
    print("Train loss: {}".format(tr_loss / nb_tr_steps))
    #模型评估
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("Validation Accuracy: {}".format(eval_accuracy / nb_eval_steps))

c:\Users\25529\anaconda3\lib\site-packages\pytorch_pretrained_bert\optimization.py:275: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:1055.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Train loss: 0.13139060938444605
Validation Accuracy: 0.9598560354374308
Train loss: 0.06520119497842784
Validation Accuracy: 0.9658776301218162
Train loss: 0.036378856597790436
Validation Accuracy: 0.9681616832779624


In [ ]:
# 保存模型参数
def save_model(the_model, PATH):
    torch.save(the_model, PATH)

In [ ]:
save_model(model, 'C:/Users/25529/Downloads/jigsaw_BERT_textAugmentation.h5')

In [ ]:
test_data = pd.read_csv('C:/Users/25529/Downloads/jigsaw/test_data.csv', encoding='ISO-8859-1')
test_data = test_data[['comment_text', 'toxic']]
test_data.head()

,comment_text,toxic
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
1,Hey... what is it..\n@ | talk .\nWhat is it......,1
2,"Bye! \n\nDon't look, come or think of comming ...",1
3,You are gay or antisemmitian? \n\nArchangel WH...,1
4,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1


In [ ]:
#提取语句并处理
sentencses=['[CLS] ' + sent + ' [SEP]' for sent in test_data.comment_text.values]
labels=test_data.toxic.values
print("第一句话:",sentencses[0])
tokenizer=BertTokenizer.from_pretrained(bert_pre_tokenizer,do_lower_case=True)
tokenized_sents=[tokenizer.tokenize(sent) for sent in sentencses]
print("tokenized的第一句话:",tokenized_sents[0])
#将分割后的句子转化成数字  word-->idx
input_ids=[tokenizer.convert_tokens_to_ids(sent) for sent in tokenized_sents]
print("转化后的第一个句子:",input_ids[0])
#PADDING
import keras
input_ids=keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print("Padding 第一个句子:",input_ids[0])
#mask
attention_masks = []
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)
print("第一个attention mask:",attention_masks[0])
#转化成tensor
test_inputs = torch.tensor(input_ids)
test_labels = torch.tensor(labels)
test_masks = torch.tensor(attention_masks)
#生成dataloader
batch_size = 64
test_data = Data.TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = Data.RandomSampler(test_data)
test_dataloader = Data.DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

第一句话: [CLS] COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK [SEP]


c:\Users\25529\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:1628: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(


tokenized的第一句话: ['[CLS]', 'cock', '##su', '##cker', 'before', 'you', 'piss', 'around', 'on', 'my', 'work', '[SEP]']
转化后的第一个句子: [101, 10338, 6342, 9102, 2077, 2017, 18138, 2105, 2006, 2026, 2147, 102]
Padding 第一个句子: [  101 10338  6342  9102  2077  2017 18138  2105  2006  2026  2147   102
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0]
第一个attention mask: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 

In [ ]:
#计算recall
def flat_recall(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return recall_score(labels_flat, pred_flat)

In [ ]:
model.eval()
test_loss, test_accuracy, test_recall = 0, 0, 0
nb_test_steps, nb_test_examples = 0, 0
for batch in test_dataloader:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)[0]
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_test_accuracy = flat_accuracy(logits, label_ids)
    tmp_test_recall = flat_recall(logits, label_ids)
    test_accuracy += tmp_test_accuracy
    test_recall += tmp_test_recall
    nb_test_steps += 1
print("Test Accuracy: {}".format(test_accuracy / nb_test_steps))
print("Test Recall: {}".format(test_recall / nb_test_steps))

Test Accuracy: 0.918996159397876
Test Recall: 0.7057049344456292
